Players dataset creation

The Fantacalcio players list is manually downloaded from https://www.fantacalcio.it/quotazioni-fantacalcio

Here, the players database is generated, by merging the Fantacalcio list to stats downloaded from http://fbref.com

In [1]:
import pandas as pd

Load fbref data for outfield players and goalkeepers.

Generate fbref player list, adding player surname (with special characters replaced to normal ones)

In [2]:
rcsv = pd.read_csv('fbref_data/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [3]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

keepers_ID = len(outfield_players)

players

,player,team
0,James Abankwah,Udinese
1,Oliver Abildgaard,Hellas Verona
2,Tammy Abraham,Roma
3,Christian Acella,Cremonese
4,Francesco Acerbi,Inter
...,...,...
617,Pietro Terracciano,Fiorentina
618,Martin Turk,Sampdoria
619,Guglielmo Vicario,Empoli
620,Jeroen Zoet,Spezia


In [4]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [5]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [6]:
print(players[['player', 'surname']].to_string())

                         player           surname
0                James Abankwah          Abankwah
1             Oliver Abildgaard        Abildgaard
2                 Tammy Abraham           Abraham
3              Christian Acella            Acella
4              Francesco Acerbi            Acerbi
5                   Yacine Adli              Adli
6              Michel Aebischer         Aebischer
7              Felix Afena-Gyan        Afena-Gyan
8                 Kevin Agudelo           Agudelo
9                      Ola Aina              Aina
10                Emanuel Aiwum             Aiwum
11       Jean-Daniel Akpa-Akpro        Akpa-Akpro
12                 Luis Alberto           Alberto
13     Agustín Álvarez Martínez          Martinez
14                 Kelvin Amian             Amian
15                 Bruno Amione            Amione
16                 Bruno Amione            Amione
17                 Ethan Ampadu            Ampadu
18               Sofyan Amrabat           Amrabat


Replace the surname for some specific players, according to config/name_fix.txt file.

This is done for players for which the decoded fbref surname doesn't correspond to Fantacalcio list.

In [7]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix



Ostigard
Kim
Hojlund
Gytkjaer
Augusto
Maehle
Kjaer
Djuricic
Djuric
Cabral
Alvarez


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Monza
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


Load players from Fantacalcio list.

In [8]:
fc_data = pd.read_excel('fantacalcio/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players['surname'][i] = spl[-2]
        fc_players['initial'][i] = spl[-1][0]
    else:
        fc_players['surname'][i] = spl[-1]
        fc_players['initial'][i] = ''
    
fc_players



C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2263921821.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-1]
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2263921821.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['initial'][i] = ''
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2263921821.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-2]


,id,r,name,team,surname,initial
0,572,P,Meret,Napoli,Meret,
1,2814,P,Provedel,Lazio,Provedel,
2,4964,P,Vicario,Empoli,Vicario,
3,453,P,Szczesny,Juventus,Szczesny,
4,2134,P,Falcone,Lecce,Falcone,
...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,
539,5837,A,Voelkerling Persson,Lecce,Persson,
540,6113,A,Montevago,Sampdoria,Montevago,
541,6143,A,Krollis,Spezia,Krollis,


Associate players from Fantacalcio list to ID for FBref data.

In [9]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players['fb_ID'][i] = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players['fb_ID'][i] = j
                
        

C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2200947104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = -1
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2200947104.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


Print players for which the association failed.

Most of them are players who didn't play a single Serie A game this season with their team. If that is the case, and there is data from their previous team, that is taken here.

Others are ones for which the FBRef surname doesn't correspond to Fantacalcio one.


For example, Cabral is Arthur for FBref.

Correction is made in the name_fix code above.

In [10]:
exceptions = ['pellegrini', 'berardi', 'romagnoli'] # exceptions for such players that have the same surname as others (Berardi A., Luca Pellegrini)

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        found = False
        for j in range(players.shape[0]):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                if(not(players['surname'][j].lower() in exceptions)):
                    if((fc_players['r'][i] == 'P') == (j >= keepers_ID)):
                        fc_players['fb_ID'][i] = j
                        found = True
        if(found):
            print(fc_players['name'][i] + ' from previous team stats')
        else:
            print(fc_players['name'][i] + ' not found')

Mirante not found
Sarr M. not found
Lamanna not found
Ujkani not found
Berisha not found
Padelli not found
Cordaz not found
Pinsoglio not found
Fiorillo not found
Rossi F. not found
Ravaglia F. from previous team stats
Brancolini not found
Bleve not found
Berardi A. not found
Russo A. not found
Gemello not found
Boer not found
Adamonis not found
Marfella not found
Piana not found
Bagnolini not found
Svilar not found
Sorrentino A. not found
Ciezkowski not found
Saro not found
Vasquez D. not found
Bereszynski from previous team stats
Aiwu not found
Radu from previous team stats
Paletta not found
Fares not found
Romagna not found
Amey not found
Buta not found
Guessand A. not found
Guarino not found


C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\362391242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


Machin not found
Akpa Akpro not found
Galdames not found
Darboe not found
Urbanski not found
Bertini not found
Trimboli not found
Samek not found
Degli Innocenti not found
Faticanti not found
Oddei not found
Kaio Jorge not found
Vivaldo not found


In [11]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,572,P,Meret,Napoli,Meret,,596
1,2814,P,Provedel,Lazio,Provedel,,607
2,4964,P,Vicario,Empoli,Vicario,,619
3,453,P,Szczesny,Juventus,Szczesny,,615
4,2134,P,Falcone,Lecce,Falcone,,589
...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,146
539,5837,A,Voelkerling Persson,Lecce,Persson,,543
540,6113,A,Montevago,Sampdoria,Montevago,,353
541,6143,A,Krollis,Spezia,Krollis,,278


Populate players dataset with stats from FBref, for outfield players and goalkeepers

In [12]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            

C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [13]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [14]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]

C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_3044\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [15]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,572,P,Meret,Napoli,Meret,,596,26-040,1997,0,...,26.3,189,20.1,26.9,315,10,3.2,35,1.13,17.1
1,2814,P,Provedel,Lazio,Provedel,,607,29-045,1994,0,...,33.3,166,36.1,34.7,422,18,4.3,46,1.44,16.0
2,4964,P,Vicario,Empoli,Vicario,,619,26-206,1996,0,...,33.4,142,49.3,42.6,504,28,5.6,15,0.60,10.7
3,453,P,Szczesny,Juventus,Szczesny,,615,33-013,1990,0,...,34.4,119,49.6,41.4,312,9,2.9,19,0.85,15.1
4,2134,P,Falcone,Lecce,Falcone,,589,28-019,1995,0,...,41.8,228,77.6,52.6,452,22,4.9,36,1.13,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,5512,A,De Luca,Sampdoria,Luca,,146,24-288,1998,2,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
539,5837,A,Voelkerling Persson,Lecce,Persson,,543,20-106,2003,7,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
540,6113,A,Montevago,Sampdoria,Montevago,,353,20-044,2003,6,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
541,6143,A,Krollis,Spezia,Krollis,,278,21-185,2001,2,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Load votes database, to add data to players database (mean vote and its standard deviation)

In [16]:
import numpy as np

votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)

Compute the average Serie A Goal Keeper mean vote and vote std

In [17]:
min_votes = 6

perf_df_P = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]): 
    if(fc_players.loc[i]['r'] == 'P'):
        v = np.array([])
        for j in range(votes.shape[0]):
            if(fc_players['name'][i] == votes['player'][j]):
                v = np.append(v, votes['vote'][j])

        if(v.shape[0] >= min_votes - 1):
            row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df_P.columns)
            perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


print(perf_df_P.mean())

perf_df_P


vote_avg    6.219162
vote_std    0.463073
dtype: float64


,vote_avg,vote_std
0,6.193548,0.414990
1,6.296875,0.430468
2,6.440000,0.382623
3,6.108696,0.328254
4,6.250000,0.530330
5,6.265625,0.414284
6,6.000000,0.586302
7,6.142857,0.349927
8,6.294118,0.455645
9,6.031250,0.431884


Add to players data their mean vote (and its standard deviation).

For players who don't have a minimum amount of games, more data to reach this value is computed, according to the average Serie A player vote (and std). For goalkeepers, this values are different.


In [ ]:
min_votes = 6

#outfield players
mean_def = 6
std_def = 0.58

#goalkeepers
mean_def_P = 6.22
std_def_P = 0.43


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    mean_def_i = mean_def
    std_def_i = std_def
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

In [ ]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

In [ ]:
fc_players.columns[123]

For goalkeepers who didn't play a miminum amount of games, data is weightly averaged with one of the main goalkeeper of their same team.

In [ ]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

In [ ]:
fc_players = fc_players_newgk

fc_players

Save to file.

In [ ]:
fc_players.to_excel('mid_outputs/players_stats.xlsx')